In [1]:
import pandas as pd
import requests

url_english = "https://drive.google.com/uc?export=download&id=1J5zhBA3qvKFPbADziKRv1xBQ9A_jMdRs"
text = requests.get(url_english).text
lines = text.splitlines()
df_english = pd.DataFrame({"text": lines})
url_indo = "https://drive.google.com/uc?export=download&id=1_jMC6ImrPz2KJzj4aWFdTiPS3_2-DcG6"
text = requests.get(url_indo).text
lines = text.splitlines()
df_indo = pd.DataFrame({"text": lines})
url_main = "https://drive.google.com/uc?export=download&id=1xisf1vNV4bqCmqfP4_SFNIOYT45K19yP"
df_main = pd.read_excel(url_main)
df_main.head(10)

,project_id,row_index,english_sentence,indonesian_sentence,similarity_score
0,7973,0,"In 2010, South Korea experienced a particularl...","Pada 2010, Korea Selatan mengalami musim dingi...",0.949540
1,7973,0,People couldn't activate their smartphones whi...,Orang-orang tidak dapat menyalakan ponsel pint...,0.722667
2,7973,0,"So, what could sausages do that gloves couldn’t?","Jadi, apa yang bisa sosis lakukan tapi sarung ...",0.714679
3,7973,0,"In other words, how do touchscreens actually w...","Dengan kata lain, bagaimana sebenarnya kerja l...",0.819390
4,7973,0,"In 1965, the first ever touchscreen was invent...","Pada 1965, pertama kali layar sentuh diciptaka...",0.851720
5,7973,0,"However, the technology was too unwieldy and e...","Akan tetapi, teknologi tersebut terlalu susah ...",0.891138
6,7973,0,"Over the following decades, engineers further ...","Berpuluh-puluh tahun kemudian, para insinyur m...",0.809739
7,7973,0,"Soon, resistive touchscreens dominated the mar...","Dengan segera, layar sentuh resistif akhirnya ...",0.773305
8,7973,0,"But then, in 2007, Apple released the first iP...","Tapi kemudian, pada 2007, Apple meluncurkan iP...",0.986689
9,7973,0,"It was a breakthrough, yet it functioned using...","Itu memang sebuah terobosan, namun dasarnya ma...",0.826725


In [2]:
filtered_df = df_main[ df_main["similarity_score"] > 0.6 ].reset_index(drop=True)

In [3]:
from datasets import Dataset

dataset = Dataset.from_pandas(filtered_df[["english_sentence", "indonesian_sentence"]])

dataset = dataset.rename_columns({
    "english_sentence": "en",
    "indonesian_sentence": "id"
})

dataset = dataset.map(lambda x: {"translation": {"en": x["en"], "id": x["id"]}})
dataset = dataset.remove_columns(["en", "id"])

dataset


Map:   0%|          | 0/5965 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 5965
})

In [4]:
dataset = dataset.train_test_split(test_size=0.05)

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-en-id"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-id. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/796k [00:00<?, ?B/s]

...

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1124 01:24:01.620000 30356 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/291M [00:00<?, ?B/s]

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_main[["english_sentence", "indonesian_sentence"]])

# split train/validation
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_ds = dataset["train"]
val_ds   = dataset["test"]


In [7]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "out_proj"
    ],
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 73,897,984 || trainable%: 1.5963


In [8]:
max_length = 128

def preprocess(batch):
    # source EN
    model_inputs = tokenizer(
        batch["english_sentence"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )

    # target ID
    labels = tokenizer(
        batch["indonesian_sentence"],
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

train_tok = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
val_tok   = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)

print(train_tok[0].keys())

Map:   0%|          | 0/11016 [00:00<?, ? examples/s]

Map:   0%|          | 0/580 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [9]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./helsinki-lora-run",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=5,
    warmup_steps=100,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    # evaluation_strategy="epoch",
    predict_with_generate=True,
    bf16=True,          # ← TURN ON
    fp16=False          # ← TURN OFF
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    tokenizer=tokenizer
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30356\499783332.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
50,8.704500
100,2.804500
150,2.382500
200,2.209600
250,2.084200
300,2.026500
350,1.997100
400,1.981200
450,1.979700
500,1.889000


TrainOutput(global_step=3445, training_loss=1.8283285619904583, metrics={'train_runtime': 1262.8154, 'train_samples_per_second': 43.617, 'train_steps_per_second': 2.728, 'total_flos': 1917022749327360.0, 'train_loss': 1.8283285619904583, 'epoch': 5.0})

In [11]:
trainer.save_model("helsinki-lora-finetuned")
tokenizer.save_pretrained("helsinki-lora-finetuned")
model.save_pretrained("helsinki-lora-finetuned")